In [3]:
!pip install torchtext

     |████████████████████████████████| 8.0 MB 3.1 MB/s eta 0:00:01
     |████████████████████████████████| 881.9 MB 267 bytes/s a 0:00:01     |███████▊                        | 211.4 MB 45.4 MB/s eta 0:00:15     |█████████▌                      | 260.3 MB 9.1 MB/s eta 0:01:09     |███████████                     | 304.1 MB 553 kB/s eta 0:17:24     |██████████████████▋             | 513.3 MB 22.8 MB/s eta 0:00:17     |████████████████████████████    | 771.5 MB 21.7 MB/s eta 0:00:06
ERROR: torchvision 0.7.0 has requirement torch==1.6.0, but you'll have torch 1.10.0 which is incompatible.
  Attempting uninstall: torch
    Found existing installation: torch 1.6.0
    Uninstalling torch-1.6.0:
      Successfully uninstalled torch-1.6.0
You should consider upgrading via the '/home/pallabee/.virtualenvs/keras/bin/python -m pip install --upgrade pip' command.


In [12]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.datasets import Multi30k
from torchtext.legacy.data import Field, BucketIterator

import numpy as np
import pandas as pd
import spacy, random

In [10]:
#Loading SpaCy's vocabulary for our desired languages.
# !python -m pip install spacy
# !python -m spacy download en 
# !python -m spacy download de

⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
     |████████████████████████████████| 13.6 MB 16.0 MB/s eta 0:00:01
You should consider upgrading via the '/home/pallabee/.virtualenvs/keras/bin/python -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ As of spaCy v3.0, shortcuts like 'de' are deprecated. Please use the
full pipeline package name 'de_core_news_sm' instead.
     |████████████████████████████████| 18.8 MB 16.8 MB/s eta 0:00:01


You should consider upgrading via the '/home/pallabee/.virtualenvs/keras/bin/python -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')


In [15]:
spacy_german = spacy.load('de_core_news_sm')
spacy_english = spacy.load('en_core_web_sm')

In [16]:
def tokenize_german(text):
    return [token.text for token in spacy_german.tokenizer(text)]

In [17]:
def tokenize_english(text):
    return [token.text for token in spacy_english.tokenizer(text)]

In [21]:
german = Field(tokenize=tokenize_german, lower = True, init_token="<sos>", eos_token="<eos>")

In [22]:
english = Field(tokenize=tokenize_english, lower = True, init_token="<sos>", eos_token="<eos>")